In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Adding Our Stocks Data to a Pandas DataFrame


In [4]:
my_coloumns = ['Ticker' ,"Price" , 'Market Capatilization','No Of Shares to Buy']
final_data = pd.DataFrame(columns = my_coloumns)


## 1 Parsing Our API Call
## 2 Looping Through The Tickers in Our List of Stocks
## 3 Using Batch API Calls to Improve Time Requirement



In [5]:
def sublists(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [6]:
groups = list(sublists(stocks['Ticker'],100))


In [7]:
symbol_str = []
for i in range(0,len(groups)):
    symbol_str.append(','.join(groups[i]))
final_data = pd.DataFrame(columns = my_coloumns)
for st in symbol_str:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable//stock/market/batch/?types=quote&symbols={st}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in st.split(','):
        final_data = final_data.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            "N/A"
        ],
        index = my_coloumns
        ),
        ignore_index = True
    )

,Ticker,Price,Market Capatilization,No Of Shares to Buy
0,A,127.550,38335303239,N/A
1,AAL,24.561,16414633697,N/A
2,AAP,180.994,12120806246,N/A
3,AAPL,127.360,2208751772731,N/A
4,ABBV,116.200,204981776359,N/A
...,...,...,...,...
500,YUM,108.980,33213334401,N/A
501,ZBH,164.310,34312984409,N/A
502,ZBRA,480.230,25813126411,N/A
503,ZION,59.125,9288086180,N/A


## Calculating the Number of Shares to Buy



In [22]:
portfolio_size = input('Entert the initial value of your portfolio:')
try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number please enter a number")
    portfolio_size = input('Entert the value of your portfolio:')
    val = float(portfolio_size)
    

Entert the initial value of your portfolio:1000000
1000000.0


In [23]:
position_size = val/len(final_data.index)
for i in range(len(final_data.index)):
    final_data.loc[i,"No Of Shares to Buy"] = math.floor(position_size/final_data["Price"][i])


,Ticker,Price,Market Capatilization,No Of Shares to Buy
0,A,127.550,38335303239,15
1,AAL,24.561,16414633697,80
2,AAP,180.994,12120806246,10
3,AAPL,127.360,2208751772731,15
4,ABBV,116.200,204981776359,17
...,...,...,...,...
500,YUM,108.980,33213334401,18
501,ZBH,164.310,34312984409,12
502,ZBRA,480.230,25813126411,4
503,ZION,59.125,9288086180,33


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

### Initializing our XlsxWriter Object

In [24]:
writer = pd.ExcelWriter('recommended_trades.xlsx' , engine='xlsxwriter')
final_data.to_excel(writer,sheet_name = 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [26]:
bg_Color = '#6fc8e3'
font_Color = '#000000'
string_format =  writer.book.add_format({
    'font_color':font_Color,
    'bg_color':bg_Color,
    'border':1
}
)
dollor_format =  writer.book.add_format({
    'num_format':'$0.00',
    'font_color':font_Color,
    'bg_color':bg_Color,
    'border':1
}
)
integer_format =  writer.book.add_format({
    'num_format':'0',
    'font_color':font_Color,
    'bg_color':bg_Color,
    'border':1
}
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [27]:
formats = {
    'A':['Ticker',string_format],
    'B':['Price',dollor_format],
    'C':['Market Capatilization',dollor_format],
    'D':['No Of Shares to Buy',integer_format]
}

for i in formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{i}:{i}', 18, formats[i][1])
    writer.sheets['Recommended Trades'].write(f'{i}1' ,formats[i][0] , string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [28]:
writer.save()